# 注意力机制计算

- 需要指定三个输入Q(query),K(key),V(value)然后通过计算公式得到注意力结果
- 这个结果带边query在key和value作用下的注意力表示
- 当q=k=v称作自注意力机制

- 当注意力权重矩阵和v都是三维张量且第一维带边batch_size时,则做bmm运算，bmm是一种特殊的张量乘法运算

# BMM运算示例

In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [2]:
mat1 = torch.randn(10,3,4)
mat2 = torch.randn(10,4,5)
res = torch.bmm(mat1,mat2)
res.size()

torch.Size([10, 3, 5])

# 注意力机制示例

In [3]:
class Atten(nn.Module):
    def __init__(self,query_size,key_size,value_size1,value_size2,output_size):
        """
            初始化函数中有5个参数
            query_size代表qury的最后一维大小
            key_size表示key最后一维大小
            value_size1代表value的倒数第二维度大小
            value_size2代表value的倒数第一维度大小
            output_size输出最后一维大小
        """
        super(Atten,self).__init__()
        self.query_size = query_size
        self.key_size = key_size
        self.value_size1 = value_size1
        self.value_size2 = value_size2
        self.output_size = output_size
        
        self.attn = nn.Linear(self.query_size + self.key_size,value_size1)
        self.attn_combine = nn.Linear(self.query_size + value_size2,output_size)
        
    def forward(self,Q,K,V):
        """
            forward函数的三个输入分别为Q，K，V
            输入Attention机制的张量都是三维，因此也加设qkv都是三维张量
        """
        # 将qk进行纵轴拼接，做一次线性变换，最后使用softmax处理获得结果
        attn_weights = F.softmax(self.attn(torch.cat((Q[0],K[0]),1)),dim=1)
        
        # 然后进行第一步后半部分，将得到的权重矩阵与v做乘法
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),V)
        
        # 先降维，根据第一步计算方法，将q与第一步计算结果进行拼接
        output = torch.cat((Q[0],attn_applied[0]),1)
        
        # 最后，使用线性层在第三步结果上做一个线性变换并扩展维度得到输出
        # 要保证输出也是三维张量，因此需要使用 unsqueeze(0)扩展维度
        output = self.attn_combine(output).unsqueeze(0)
        
        return output,attn_weights

In [4]:
query_size = 32
key_size = 32
value_size1 = 32
value_size2 = 64
ouput_size = 64

attn = Atten(query_size,key_size,value_size1,value_size2,ouput_size)

In [5]:
Q = torch.randn(1,1,32)
K = torch.randn(1,1,32)
V = torch.randn(1,32,64)
ouput = attn(Q,K,V)

In [6]:
print(ouput[0])
print(ouput[0].size())

tensor([[[-1.4020e-01, -1.9777e-02, -5.9354e-01, -4.3420e-01, -2.4355e-01,
           4.2465e-01, -1.6861e-01,  1.1785e-01, -2.9805e-01, -1.3324e-01,
           4.7061e-01, -2.8108e-01,  8.6837e-02,  1.3431e-01,  3.1195e-01,
           7.1486e-01, -2.6822e-01, -1.1985e-01, -4.6764e-01,  1.8691e-01,
          -5.0815e-01,  1.7527e-01,  7.0029e-02, -3.4055e-01, -2.1331e-01,
           3.2258e-01, -1.3103e-01,  6.3416e-02, -5.0219e-01,  1.2042e-01,
           1.5124e-01,  2.7970e-01,  2.6234e-01, -1.5407e-02, -2.4809e-01,
           4.8951e-03,  2.6403e-01,  5.3912e-02,  1.6241e-01, -8.9987e-02,
          -4.6360e-01,  2.3851e-02, -4.0053e-02, -2.8955e-01, -1.1405e-01,
           1.1497e-01, -2.0218e-04, -7.6098e-03,  3.6800e-01, -2.3736e-01,
          -2.4199e-01,  5.4815e-01, -1.0892e-01, -3.5993e-01,  2.9366e-01,
          -4.4473e-01, -1.9299e-02, -2.5384e-02, -3.2065e-01, -5.3196e-01,
          -2.4980e-01, -1.8209e-01, -2.9858e-01,  1.7556e-01]]],
       grad_fn=<UnsqueezeBackward0>

In [7]:
print(ouput[1])
print(ouput[1].size())

tensor([[0.0161, 0.0205, 0.0287, 0.0253, 0.0196, 0.0270, 0.0211, 0.0382, 0.0239,
         0.0288, 0.0205, 0.0599, 0.0296, 0.0204, 0.0290, 0.0354, 0.0248, 0.0204,
         0.1460, 0.0190, 0.0099, 0.0126, 0.0415, 0.0283, 0.0955, 0.0305, 0.0198,
         0.0192, 0.0187, 0.0370, 0.0081, 0.0246]], grad_fn=<SoftmaxBackward>)
torch.Size([1, 32])
